In [7]:
import random
import pandas as pd
import numpy as np
from faker import Faker
from datetime import timedelta, datetime

## --- Generación de empleados.csv ---

In [8]:
fake = Faker('es_ES')

In [9]:
num_empleados = 150
departamentos = ["Ventas", "Marketing", "IT", "Recursos Humanos", "Finanzas", "Operaciones"]
puestos_por_departamento = {
    "Ventas": ["Representante de Ventas", "Gerente de Ventas", "Ejecutivo de Cuentas"],
    "Marketing": ["Especialista en Marketing", "Analista de Marketing", "Community Manager"],
    "IT": ["Desarrollador Front-end", "Desarrollador Back-end", "Administrador de Sistemas", "Analista de Datos"],
    "Recursos Humanos": ["Especialista en RRHH", "Técnico de Selección", "Responsable de Formación"],
    "Finanzas": ["Contador", "Analista Financiero", "Tesorero"],
    "Operaciones": ["Coordinador de Operaciones", "Especialista en Logística", "Asistente de Producción"],
}

empleados_data = []
for i in range(1, num_empleados + 1):
    nombre = fake.name()
    correo = f"{nombre.split()[0].lower()}.{nombre.split()[1].lower()}@empresa.com"
    edad = random.randint(22, 55)
    genero = random.choice(["Masculino", "Femenino", "Otro"])
    departamento = random.choice(departamentos)
    puesto = random.choice(puestos_por_departamento[departamento])
    fecha_contratacion = datetime.now() - timedelta(days=random.randint(30, 1825)) # Hasta 5 años de antigüedad
    antiguedad_meses = (datetime.now() - fecha_contratacion).days // 30
    nivel_educacion = random.choice(["Bachillerato", "Grado", "Máster", "Doctorado"])
    empleados_data.append([i, nombre, departamento, puesto, fecha_contratacion.strftime('%Y-%m-%d'), antiguedad_meses, correo, edad, genero, nivel_educacion])

empleados_df = pd.DataFrame(empleados_data, columns=['id_empleado', 'nombre', 'departamento', 'puesto', 'fecha_contratacion', 'antiguedad_meses', 'correo_electronico', 'edad', 'genero', 'nivel_educacion'])
empleados_df.to_csv('../data/empleados.csv', index=False)

## --- Generación de calendario_laboral.csv ---

In [10]:
start_date = datetime(2023, 1, 1)
end_date = datetime(2025, 12, 31)
dates = pd.date_range(start=start_date, end=end_date, freq='D')
calendario_data = []

# Definir festivos para 2023, 2024 y 2025 (España y algunos locales de Jerez - ejemplo)
festivos = {
    2023: ["2023-01-01", "2023-01-06", "2023-02-28", "2023-04-06", "2023-04-07", "2023-05-01", "2023-08-15", "2023-10-12", "2023-11-01", "2023-12-06", "2023-12-08", "2023-12-25"],
    2024: ["2024-01-01", "2024-01-06", "2024-02-28", "2024-03-28", "2024-03-29", "2024-05-01", "2024-08-15", "2024-10-12", "2024-11-01", "2024-12-06", "2024-12-08", "2024-12-25"],
    2025: ["2025-01-01", "2025-01-06", "2025-02-28", "2025-04-17", "2025-04-18", "2025-05-01", "2025-08-15", "2025-10-12", "2025-11-01", "2025-12-06", "2025-12-08", "2025-12-25"],
}

for date in dates:
    fecha_str = date.strftime('%Y-%m-%d')
    anio = date.year
    es_festivo = fecha_str in festivos.get(anio, [])
    dia_semana = date.strftime('%A')
    mes = date.month
    if 3 <= mes <= 5:
        temporada = "Primavera"
    elif 6 <= mes <= 8:
        temporada = "Verano"
    elif 9 <= mes <= 11:
        temporada = "Otoño"
    else:
        temporada = "Invierno"
    calendario_data.append([fecha_str, es_festivo, dia_semana, temporada])

calendario_df = pd.DataFrame(calendario_data, columns=['fecha', 'es_festivo', 'dia_semana', 'temporada'])
calendario_df.to_csv('../data/calendario_laboral.csv', index=False)

print("Archivo calendario_laboral.csv generado.")

Archivo calendario_laboral.csv generado.


## --- Generación de politicas_vacaciones.csv ---

In [11]:
politicas_data = [
    ["General", 15, None, None, None, False, None],
    ["Ventas", 10, 6, None, None, True, 2],
    ["Marketing", 12, 3, datetime(datetime.now().year, 8, 1).strftime('%Y-%m-%d'), datetime(datetime.now().year, 8, 15).strftime('%Y-%m-%d'), False, 1],
    ["IT", 20, None, None, None, False, 3],
    ["Recursos Humanos", 10, 12, None, None, True, 1],
    ["Finanzas", 15, 6, datetime(datetime.now().year, 12, 20).strftime('%Y-%m-%d'), datetime(datetime.now().year, 1, 7).strftime('%Y-%m-%d'), False, 1],
    ["Operaciones", 18, None, None, None, True, 2],
]

politicas_df = pd.DataFrame(politicas_data, columns=['departamento', 'max_dias_consecutivos', 'antiguedad_minima_meses', 'periodo_bloqueo_inicio', 'periodo_bloqueo_fin', 'prioridad_antiguedad', 'max_empleados_simultaneos'])
politicas_df.to_csv('../data/politicas_vacaciones.csv', index=False)

print("Archivo politicas_vacaciones.csv generado.")

Archivo politicas_vacaciones.csv generado.


## --- Generación de solicitudes_vacaciones.csv (con carga de políticas) ---

In [12]:
# Cargar el DataFrame de políticas de vacaciones
try:
    politicas_df = pd.read_csv('../data/politicas_vacaciones.csv')
    politicas_por_departamento = politicas_df.set_index('departamento').to_dict('index')
except FileNotFoundError:
    print("Error: El archivo politicas_vacaciones.csv no se encontró. Asegúrate de generarlo primero.")
    exit()

num_solicitudes = 10000
solicitudes_data = []

# Simulación del estado de vacaciones por departamento (para evitar solapamientos excesivos)
vacaciones_programadas = {dep: [] for dep in empleados_df['departamento'].unique()}

for i in range(1, num_solicitudes + 1):
    id_empleado = random.choice(empleados_df['id_empleado'])
    empleado = empleados_df[empleados_df['id_empleado'] == id_empleado].iloc[0]
    departamento_empleado = empleado['departamento']
    antiguedad_meses_empleado = empleado['antiguedad_meses']

    fecha_solicitud = datetime.now() - timedelta(days=random.randint(10, 365))
    fecha_inicio = fecha_solicitud + timedelta(days=random.randint(5, 90))
    duracion_dias = random.randint(1, 20)
    fecha_fin = fecha_inicio + timedelta(days=duracion_dias - 1)
    periodo_anio = fecha_inicio.year
    mes_solicitud = fecha_solicitud.month
    dia_semana_solicitud = fecha_solicitud.strftime('%A')

    estado = "Aceptada"
    motivo_rechazo = np.nan

    # --- Lógica de Decisión basada en políticas del CSV ---
    rechazo = False
    motivos_rechazo_lista = []

    politica_departamento = politicas_por_departamento.get(departamento_empleado)
    politica_general = politicas_por_departamento.get("General", {}) # Obtener políticas generales por si faltan específicas

    max_dias = politica_departamento.get('max_dias_consecutivos', politica_general.get('max_dias_consecutivos', 15))
    antiguedad_minima = politica_departamento.get('antiguedad_minima_meses')
    periodo_bloqueo_inicio_str = politica_departamento.get('periodo_bloqueo_inicio')
    periodo_bloqueo_fin_str = politica_departamento.get('periodo_bloqueo_fin')
    prioridad_antiguedad = politica_departamento.get('prioridad_antiguedad', politica_general.get('prioridad_antiguedad', False))
    max_simultaneos = politica_departamento.get('max_empleados_simultaneos', politica_general.get('max_empleados_simultaneos'))

    periodo_bloqueo_inicio = datetime.strptime(periodo_bloqueo_inicio_str, '%Y-%m-%d').date() if isinstance(periodo_bloqueo_inicio_str, str) else None
    periodo_bloqueo_fin = datetime.strptime(periodo_bloqueo_fin_str, '%Y-%m-%d').date() if isinstance(periodo_bloqueo_fin_str, str) else None

    # 1. Excede la duración máxima consecutiva
    if duracion_dias > max_dias:
        rechazo = True
        motivos_rechazo_lista.append(f"Duración de vacaciones excede el límite permitido de {max_dias} días.")

    # 2. No cumple la antigüedad mínima (si aplica)
    if antiguedad_minima is not None and antiguedad_meses_empleado < antiguedad_minima:
        rechazo = True
        motivos_rechazo_lista.append(f"No se cumple la antigüedad mínima de {antiguedad_minima} meses.")

    # 3. Solicitud dentro del periodo de bloqueo (si aplica)
    if periodo_bloqueo_inicio and periodo_bloqueo_fin:
        fecha_inicio_solicitud = fecha_inicio.date()
        fecha_fin_solicitud = fecha_fin.date()
        if (periodo_bloqueo_inicio <= fecha_inicio_solicitud <= periodo_bloqueo_fin) or \
           (periodo_bloqueo_inicio <= fecha_fin_solicitud <= periodo_bloqueo_fin) or \
           (fecha_inicio_solicitud <= periodo_bloqueo_inicio and fecha_fin_solicitud >= periodo_bloqueo_fin):
            # Considerar antigüedad para permitir algunas excepciones en periodos de bloqueo
            if antiguedad_meses_empleado < 12 and random.random() < 0.7: # Ejemplo de lógica para excepciones
                rechazo = True
                motivos_rechazo_lista.append("Solicitud dentro del periodo de bloqueo de vacaciones.")

    # 4. Exceso de empleados de vacaciones simultáneamente en el departamento (si se define el límite)
    if max_simultaneos is not None:
        solapamientos = 0
        for emp_id_programado, inicio, fin in vacaciones_programadas[departamento_empleado]:
            if (fecha_inicio <= fin) and (fecha_fin >= inicio):
                solapamientos += 1
        if solapamientos >= max_simultaneos:
            # Considerar prioridad por antigüedad
            empleados_solapados = [emp_id_prog for emp_id_prog, inicio, fin in vacaciones_programadas[departamento_empleado] if (fecha_inicio <= fin) and (fecha_fin >= inicio)]
            antiguedades_solapados = [empleados_df[empleados_df['id_empleado'] == emp_id]['antiguedad_meses'].iloc[0] for emp_id in empleados_solapados]
            if prioridad_antiguedad and antiguedad_meses_empleado <= min(antiguedades_solapados + [float('inf')]):
                rechazo = True
                motivos_rechazo_lista.append(f"Se supera el número máximo de {max_simultaneos} empleados de vacaciones simultáneamente en el departamento.")
            elif not prioridad_antiguedad and random.random() < 0.5: # Si no hay prioridad, rechazo aleatorio si se supera el límite
                rechazo = True
                motivos_rechazo_lista.append(f"Se supera el número máximo de {max_simultaneos} empleados de vacaciones simultáneamente en el departamento.")


    # Decidir el estado final y el motivo
    if rechazo:
        estado = "Rechazada"
        motivo_rechazo = random.choice(motivos_rechazo_lista) if motivos_rechazo_lista else "Motivo no especificado."
    else:
        # Si se acepta, registrar las vacaciones programadas para futuras decisiones (incluyendo el id del empleado)
        vacaciones_programadas[departamento_empleado].append((id_empleado, fecha_inicio, fecha_fin))

    solicitudes_data.append([i, id_empleado, fecha_inicio.strftime('%Y-%m-%d'), fecha_fin.strftime('%Y-%m-%d'), duracion_dias, fecha_solicitud.strftime('%Y-%m-%d'), estado, motivo_rechazo, periodo_anio, mes_solicitud, dia_semana_solicitud])

solicitudes_df = pd.DataFrame(solicitudes_data, columns=['id_solicitud', 'id_empleado', 'fecha_inicio', 'fecha_fin', 'duracion_dias', 'fecha_solicitud', 'estado', 'motivo_rechazo', 'periodo_anio', 'mes_solicitud', 'dia_semana_solicitud'])
solicitudes_df.to_csv('../data/solicitudes_vacaciones_con_politicas.csv', index=False)

print("Archivo solicitudes_vacaciones_con_politicas.csv generado.")

Archivo solicitudes_vacaciones_con_politicas.csv generado.
